In [ ]:
from dataset_preprocessing import ProductionModeDataset

import torch
from sklearn.inspection import permutation_importance
import numpy as np
import uproot
import torch.nn as nn
from matplotlib import pyplot as plt

from Classifier_module import Classifier


In [3]:
######### configure stuff here
file = "/depot/darkmatter/data/jupyterhub/Physics_Undergrads/Steve/things/all_1.root"
input_variables = uproot.open(file)["ttBar_treeVariables_step8;5;1"].keys()
configDir = "."
two_layer = True

In [6]:
############ Load data in here

#########################

PATH = "./models/twoLayerModel_2.pt"
if two_layer:
#     model = torch.load(configDir + "/models/twoLayerModel.pt")
    model = Classifier()
    model.load_state_dict(torch.load(PATH))
    model.eval()

NameError: name 'Classifier' is not defined

In [36]:
model.eval()

Sequential(
  (0): Linear(in_features=80, out_features=256, bias=True)
  (1): LeakyReLU(negative_slope=0.2, inplace=True)
  (2): Linear(in_features=256, out_features=3, bias=True)
  (3): LeakyReLU(negative_slope=0.2, inplace=True)
)

In [55]:
data_object = ProductionModeDataset(file)
x = data_object.get_eval_data()


['ttBar_treeVariables_step8;5;1']
['lb_delta_eta', 'lbbar_delta_eta', 'lnu_delta_eta', 'lnubar_delta_eta', 'lbarb_delta_eta', 'lbarbbar_delta_eta', 'lbarnu_delta_eta', 'lbarnubar_delta_eta', 'bnu_delta_eta', 'bnubar_delta_eta', 'bbarnu_delta_eta', 'bbarnubar_delta_eta', 'lb_delta_phi', 'lbbar_delta_phi', 'lnu_delta_phi', 'lnubar_delta_phi', 'lbarb_delta_phi', 'lbarbbar_delta_phi', 'lbarnu_delta_phi', 'lbarnubar_delta_phi', 'bnu_delta_phi', 'bnubar_delta_phi', 'bbarnu_delta_phi', 'bbarnubar_delta_phi', 'wplusb_delta_eta', 'wplusbbar_delta_eta', 'wminusb_delta_eta', 'wminusbbar_delta_eta', 'wplusb_delta_phi', 'wplusbbar_delta_phi', 'wminusb_delta_phi', 'wminusbbar_delta_phi', 'top_eta', 'top_boosted_eta', 'tbar_eta', 'tbar_boosted_eta', 'ttbar_delta_eta', 'ttbar_eta', 'llbar_delta_eta', 'bbbar_delta_eta', 'nunubar_delta_eta', 'top_phi', 'tbar_phi', 'ttbar_phi', 'ttbar_delta_phi', 'llbar_phi', 'llbar_delta_phi', 'bbbar_phi', 'bbbar_delta_phi', 'nunubar_phi', 'nunubar_delta_phi', 'l_eta', 

In [60]:
from torch.autograd import Variable

weight = x[:,81]
target = x[:,80]
target = Variable(torch.from_numpy(target).type(torch.LongTensor))
y = np.transpose(x)
y = np.delete(y, [80, 81, 82], 0)
y = np.transpose(y)
input = Variable(torch.from_numpy(y).type(torch.Tensor))

In [70]:
model(input)[0,:]

tensor([4.6072, 0.6396, 1.3612], grad_fn=<SliceBackward>)

In [72]:
target[0]

tensor(2)

In [69]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()

criterion(model(input)[0,:], target[0])

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [73]:
r = permutation_importance(model, input, target, \
                           n_repeats=30, \
                           random_state=0)

for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{input_variables[i]:<8}"
              f"{r.importances_mean[i]:.3f}"
              f" +/- {r.importances_std[i]:.3f}")


TypeError: estimator should be an estimator implementing 'fit' method, Sequential(
  (0): Linear(in_features=80, out_features=256, bias=True)
  (1): LeakyReLU(negative_slope=0.2, inplace=True)
  (2): Linear(in_features=256, out_features=3, bias=True)
  (3): LeakyReLU(negative_slope=0.2, inplace=True)
) was passed